In [1]:
from dependencies import *

_ColormakerRegistry()

In [3]:
species = ["Si", "N", "H", "C", "F", "Ar", "He"]
soap = SOAP(
    species=species,
    periodic=True,
    rcut=5,
    nmax=10,
    lmax=9,
)
soap.get_number_of_features()

15400

In [12]:
datadirs = ['../cf4_adsorb/']
adslen = 5

datalist = []
for datadir in datadirs:
    data = readStructs(datadir) #MeF, amorphous slab, DFTB
    data['processed'] = [
        convertAdsorbateToHe(
            i, len(i) - adslen, np.arange(len(i) - adslen, len(i))
        ) for i in data['geom']
    ]
    data['ads_position'] = [
        geom[-1].position[:2] for geom in data['processed']
    ]
    data['soap_indices'] = [
        [atom.index for atom in geom if np.sqrt(np.sum((atom.position[:2] - position)**2)) < 5] 
        for geom, position in zip(data['geom'], data['ads_position'])
    ]
data

,E,E_ads,geom,processed,ads_position,soap_indices
0,-17355.0308,-0.2900,"(Atom('H', [5.687816897, 11.8095906, 0.4501787...","(Atom('H', [5.687816897, 11.8095906, 0.4501787...","[5.468603081, 5.039843148]","[2, 4, 8, 13, 14, 16, 18, 20, 26, 31, 33, 34, ..."
1,-17354.9154,-0.1746,"(Atom('H', [5.687816897, 11.8095906, 0.4501787...","(Atom('H', [5.687816897, 11.8095906, 0.4501787...","[11.20257705, 1.831604262]","[4, 11, 12, 13, 27, 33, 35, 38, 41, 59, 62, 66..."
2,-17354.9170,-0.1762,"(Atom('H', [5.687816897, 11.8095906, 0.4501787...","(Atom('H', [5.687816897, 11.8095906, 0.4501787...","[11.58678762, 7.868247458]","[3, 4, 13, 15, 26, 35, 42, 47, 48, 52, 54, 55,..."
3,-17354.8291,-0.0883,"(Atom('H', [5.687816897, 11.8095906, 0.4501787...","(Atom('H', [5.687816897, 11.8095906, 0.4501787...","[8.024713031, 0.8921075379]","[11, 13, 14, 18, 27, 33, 35, 38, 39, 41, 49, 5..."
4,-17354.9063,-0.1655,"(Atom('H', [5.687816897, 11.8095906, 0.4501787...","(Atom('H', [5.687816897, 11.8095906, 0.4501787...","[1.461470922, 9.974551352]","[0, 2, 6, 9, 17, 20, 22, 24, 29, 30, 31, 43, 4..."
...,...,...,...,...,...,...
63,-17354.9204,-0.1796,"(Atom('H', [5.687816897, 11.8095906, 0.4501787...","(Atom('H', [5.687816897, 11.8095906, 0.4501787...","[10.30573163, 3.508483415]","[4, 11, 12, 13, 27, 33, 35, 38, 41, 52, 56, 58..."
64,-17354.9088,-0.1680,"(Atom('H', [5.687816897, 11.8095906, 0.4501787...","(Atom('H', [5.687816897, 11.8095906, 0.4501787...","[14.46761898, 1.568592562]","[11, 12, 13, 27, 62, 66, 76, 80, 82, 84, 90, 9..."
65,-17354.8897,-0.1489,"(Atom('H', [5.687816897, 11.8095906, 0.4501787...","(Atom('H', [5.687816897, 11.8095906, 0.4501787...","[10.78841027, 12.3959481]","[3, 5, 7, 15, 19, 21, 23, 25, 26, 28, 32, 40, ..."
66,-17354.9084,-0.1676,"(Atom('H', [5.687816897, 11.8095906, 0.4501787...","(Atom('H', [5.687816897, 11.8095906, 0.4501787...","[11.78983217, 6.857191817]","[3, 4, 13, 15, 35, 38, 42, 47, 48, 52, 54, 55,..."


In [21]:
nsamples = 2
soaps = soap.create(data['geom'][:nsamples], positions = data['soap_indices'][:nsamples])

In [22]:
soaps.shape

(217, 15400)

In [23]:
from ase.build import fcc111, add_adsorbate
slab_pure = fcc111('Al', size=(2, 2, 3), vacuum=10.0)
slab_ads = slab_pure.copy()
add_adsorbate(slab_ads, 'H', 1.5, 'ontop')
ontop_pos = slab_ads.get_positions()[-1]
add_adsorbate(slab_ads, 'H', 1.5, 'bridge')
bridge_pos = slab_ads.get_positions()[-1]
add_adsorbate(slab_ads, 'H', 1.5, 'hcp')
hcp_pos = slab_ads.get_positions()[-1]
add_adsorbate(slab_ads, 'H', 1.5, 'fcc')
fcc_pos = slab_ads.get_positions()[-1]

In [24]:
view(slab_ads)